In [12]:
from utils.read_data import get_train_targets, load_data, get_test_data, prepare_submission
from utils.generate_run_name import generate_run_name
import numpy as np
import logging
from sklearn.pipeline import Pipeline
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
import catboost as cb
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from autogluon.tabular import TabularPredictor
import logging
import pandas as pd
from sklearn.base import BaseEstimator, RegressorMixin
import pprint

In [18]:
pp = pprint.PrettyPrinter(indent=1)

In [2]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached pyfiglet-1.0.2-py3-none-any.whl (1.1 MB)
  Using cached scikit_optimize-0.9.0-py2.py3-none-any.whl (100 kB)
     |████████████████████████████████| 19.0 MB 4.2 MB/s eta 0:00:015MB/s eta 0:00:04


  Using cached pyaml-23.9.7-py3-none-any.whl (23 kB)
  Using cached databricks_cli-0.18.0-py2.py3-none-any.whl (150 kB)
  Using cached gunicorn-21.2.0-py3-none-any.whl (80 kB)
  Using cached docker-6.1.3-py3-none-any.whl (148 kB)
     |████████████████████████████████| 190 kB 99.9 MB/s eta 0:00:01
     |████████████████████████████████| 226 kB 99.2 MB/s eta 0:00:01


  Using cached sqlparse-0.4.4-py3-none-any.whl (41 kB)
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
     |████████████████████████████████| 40.1 MB 95.9 MB/s eta 0:00:01            | 552 kB 95.9 MB/s eta 0:00:01                    | 6.2 MB 95.9 MB/s eta 0:00:01███▌                 | 18.1 MB 95.9 MB/s eta 0:00:01[K     |███████████████████             | 23.8 MB 95.9 MB/s eta 0:00:01█████▍        | 29.2 MB 95.9 MB/s eta 0:00:01  | 35.7 MB 95.9 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 7.5 MB/s  eta 0:00:01
     |████████████████████████████████| 62 kB 1.2 MB/s  eta 0:00:01
  Using cached smmap-5.0.1-py3-none-any.whl (24 kB)


  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.1
    Uninstalling pyarrow-14.0.1:
      Successfully uninstalled pyarrow-14.0.1


In [2]:
drop_cols = ['time', 'elevation:m', 'fresh_snow_1h:cm', 'ceiling_height_agl:m', 'snow_density:kgm3', 
             'wind_speed_w_1000hPa:ms', 'snow_drift:idx', 'fresh_snow_3h:cm', 'is_in_shadow:idx', 'dew_or_rime:idx', 'fresh_snow_6h:cm', 'prob_rime:p'] # this second line is columns with feature importance == 0

data_a, data_b, data_c = load_data(mean=True, remove_out=True, roll_avg=True, cust_feat=True, drop_cols=drop_cols, cycle_encoding = True)
X_test_a, X_test_b, X_test_c = get_test_data(mean=True, roll_avg=True, cust_feat=True, cycle_encoding = True)

# Assuming get_train_targets is defined and returns the features and target
X_train_a, y_a = get_train_targets(data_a)
X_train_b, y_b = get_train_targets(data_b)
X_train_c, y_c = get_train_targets(data_c)

In [44]:
predictor = TabularPredictor.load(
    path='/cluster/home/taheeraa/code/tdt4173-project/autogluon/Jert Mira-B',
    require_py_version_match=False,
)

Found 1 mismatches between original and current metadata:


In [60]:
import pickle

# Replace this with the path to your .pkl file
file_path = '/cluster/home/taheeraa/code/tdt4173-project/autogluon/Jert Mira-A/models/LightGBMXT/model.pkl'

# Open the file in binary read mode
with open(file_path, 'rb') as file:
    model = pickle.load(file)

# Now you can use the 'model' object

In [61]:
model

In [70]:
this_model = model.model
this_model.params

{'learning_rate': 0.05,
 'extra_trees': True,
 'num_threads': 24,
 'objective': 'regression',
 'verbose': -1,
 'metric': 'rmse',
 'seed': 0,
 'num_iterations': 10000,
 'early_stopping_round': None}

In [71]:
dir(this_model)

['_Booster__attr',
 '_Booster__boost',
 '_Booster__get_eval_info',
 '_Booster__higher_better_inner_eval',
 '_Booster__init_predictor',
 '_Booster__inner_eval',
 '_Booster__inner_predict',
 '_Booster__inner_predict_buffer',
 '_Booster__is_predicted_cur_iter',
 '_Booster__name_inner_eval',
 '_Booster__need_reload_eval_info',
 '_Booster__num_class',
 '_Booster__num_dataset',
 '_Booster__num_inner_eval',
 '_Booster__set_objective_to_none',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_free_buffer',
 '_to_predictor',
 '_train_data_name',
 'add_valid',
 'attr',
 'best_iteration',
 'best_score',
 

In [46]:
predictor.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-49.655477,1.266572,474.594725,0.008484,0.427902,2,True,12
1,LightGBMXT,-50.108285,0.177498,46.687295,0.177498,46.687295,1,True,3
2,LightGBM,-52.237785,0.160774,47.882154,0.160774,47.882154,1,True,4
3,LightGBMLarge,-52.325876,0.396332,176.754433,0.396332,176.754433,1,True,11
4,CatBoost,-53.591718,0.014859,124.369996,0.014859,124.369996,1,True,6
5,XGBoost,-56.221749,0.258444,67.121776,0.258444,67.121776,1,True,9
6,NeuralNetTorch,-58.394746,0.523484,202.842942,0.523484,202.842942,1,True,10
7,NeuralNetFastAI,-62.022147,0.050173,52.154198,0.050173,52.154198,1,True,8
8,RandomForestMSE,-62.480228,0.192984,18.190708,0.192984,18.190708,1,True,5
9,ExtraTreesMSE,-62.667320,0.184243,6.545680,0.184243,6.545680,1,True,7


In [47]:
individual_models = predictor.get_model_names()
individual_models

['KNeighborsUnif',
 'KNeighborsDist',
 'LightGBMXT',
 'LightGBM',
 'RandomForestMSE',
 'CatBoost',
 'ExtraTreesMSE',
 'NeuralNetFastAI',
 'XGBoost',
 'NeuralNetTorch',
 'LightGBMLarge',
 'WeightedEnsemble_L2']

In [48]:
predictor.get_model_full_dict(inverse=False)

{}

In [28]:
pp.pprint(models_info)

{}


In [3]:
class AutoGluonWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, path):
        self.path = path
        self.model = 

    def fit(self, X, y):
        # AutoGluon's fit method is called during the initial training, 
        # so we don't need to call it here again.
        # Just return self to adhere to scikit-learn's fit method requirements.
        return self

    def predict(self, X):
        return self.model.predict(X)

In [8]:
# Now you can add your AutoGluon model to the base models
autogluon_model_a = AutoGluonWrapper("C:\\Users\\Taheera Ahmed\\code\\tdt4173-project\\autogluon\\Bolk Cubi-A")

"""
cat_boost1: bayesian_search on params given feat_eng from 07.nov
cat_boost2: from best kaggle submission
"""
base_model_a = [
    ('autogluon', autogluon_model_a),
    ('cat_boost3', cb.CatBoostRegressor(random_state=42, silent=True)),
]
# Define meta-learners for each location
meta_learner_a = LinearRegression()

    # Create the stacking regressor for the current location
stacked_model = StackingRegressor(estimators=base_model_a, final_estimator=meta_learner_a)

# Create the whole model pipeline for the current location
whole_model_pipeline = Pipeline([
    ('stacked_model', stacked_model)
])

In [9]:
whole_model_pipeline.fit(X_train_a, y_a)
predictions = whole_model_pipeline.predict(X_test_a.drop(columns=["id", "prediction", "location"]))

# Prepare submission using the predictions
prepare_submission(X_test_a, predictions, run_name=run_name)